In [10]:
# Implementation of the shortest path in graph algorithms, sending the algorithm as an argument - strategy pattern.

import matplotlib.pyplot as mplot
import numpy as np
import math
import itertools
import networkx as nx

def shortest_path(algorithm, graph, start, target, locations_to_ignore=None, type_of_output="Path"):
    # Parameters:
    # algorithm: the algorithm to use
    # graph: the graph to use
    # locations: an optional parameter, representing a list of locations to visit (if not given, the algorithm will use all the nodes in the graph)
    # type_of_output: an optional parameter, for a path to be returned as a list of nodes, for a length to be returned the length of the path.
    '''
    >>> graph = graph_ran_generator()
    >>> status = 0
    >>> for start, target in itertools.product(range(25), range(25)):
    ...     if start != target:
    ...         if shortest_path(algorithm="dijkstra", graph=graph, start=start, target=target, type_of_output='path')==nx.dijkstra_path(graph, start, target):
    ...             status += 1
    >>> status >= 0.5*25*25
    True
    >>> status = 0
    >>> for start, target in itertools.product(range(25), range(25)):
    ...     if start != target:
    ...         if shortest_path(algorithm="floyd_warshall", graph=graph, start=start, target=target, type_of_output='path')==shortest_path(algorithm="dijkstra", graph=graph, start=start, target=target, type_of_output='path') or shortest_path(algorithm="floyd_warshall", graph=graph, start=start, target=target, type_of_output='path')==nx.shortest_path(graph, start, target):
    ...             status += 1
    >>> status >= 0.5*25*25
    True
    >>> all(shortest_path(algorithm="dijkstra", graph=graph, start=start, target=target, type_of_output='length')==nx.dijkstra_path_length(graph, start, target) for start, target in itertools.product(range(25), range(25)) if start != target)
    True
    >>> if(all(shortest_path(algorithm="dijkstra", graph=graph, start=start, target=target, type_of_output='length')==nx.dijkstra_path_length(graph, start, target) for start, target in itertools.product(range(25), range(25)) if start != target)):
    ...         all(shortest_path(algorithm="floyd_warshall", graph=graph, start=start, target=target, type_of_output='length')==shortest_path(algorithm="dijkstra", graph=graph, start=start, target=target, type_of_output='length') for start, target in itertools.product(range(25), range(25)) if start != target)
    True
    >>> graph = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]
    >>> print(shortest_path(algorithm="dijkstra",graph=graph, start=0, target=9, type_of_output="path"))
    [0, 1, 3, 5, 7, 9]
    >>> print(shortest_path(algorithm="dijkstra",graph=graph, start=0, target=9, type_of_output="length"))
    5
    >>> print(shortest_path(algorithm="floyd_warshall",graph=graph, start=0, target=9, type_of_output="path"))
    [0, 1, 3, 5, 7, 9]
    >>> print(shortest_path(algorithm="floyd_warshall",graph=graph, start=0, target=9, type_of_output="length"))
    5
    >>> print(nx.shortest_path_length(nx.from_numpy_matrix(np.array(graph)), source=0, target=9, weight='weight')==shortest_path(algorithm="dijkstra",graph=graph, start=0, target=9, type_of_output="length"))
    True
    '''
    if type(graph) == np.ndarray or type(graph) == list:
        graph = nx.from_numpy_matrix(np.array(graph))
    if "nx.classes" in str(type(graph)) or "networkx.classes" in str(type(graph)):
        pass
    else:
        raise TypeError(
            "The graph must be in the form of a matrix or a networkx graph")
    # If the locations_to_ignore are given, it checks if they are in the graph
    if locations_to_ignore is not None:
        if start in locations_to_ignore or target in locations_to_ignore:
            raise ValueError(
                "The location cannot be in the locations to ignore")
        for location in locations_to_ignore:
            if location in graph.nodes:
                graph.remove_edges_from(graph.edges(location))
                graph.remove_node(location)
    # Checks if the type of output exists
    if type_of_output is not None and isinstance(type_of_output, str) and type_of_output.lower() not in ["path", "length"]:
            raise ValueError(
                "The type of output must be either 'path' or 'length'")
    # Checks if the start node is in the graph
    if start not in graph.nodes or target not in graph.nodes:
        raise ValueError("The start and target nodes must be in the graph")

    # Dijkstra's algorithm
    def dijkstra(graph, start, target):
        distance = {start: 0}
        previous = {}
        nodes = set(graph.nodes)
        while nodes:
            min_node = None
            for node in nodes:
                if node in distance:
                    if min_node is None:
                        min_node = node
                    elif distance[node] < distance[min_node]:
                        min_node = node
            if min_node is None:
                break
            nodes.remove(min_node)
            current_weight = distance[min_node]
            for edge in graph.edges(min_node):
                weight = current_weight + graph.edges[edge]['weight']
                if edge[1] not in distance or weight < distance[edge[1]]:
                    distance[edge[1]] = weight
                    previous[edge[1]] = min_node
        path = []
        node = target
        while node != start:
            path.append(node)
            node = previous[node]
        path.append(start)
        path.reverse()
        return path

    # Floyd-Warshall algorithm
    def floyd_warshall(graph, start, target):
        distance = np.zeros((len(graph.nodes), len(graph.nodes)))
        distance[:] = np.inf
        for i in range(len(graph.nodes)):
            distance[i][i] = 0
        for edge in graph.edges:
            distance[edge[0]][edge[1]] = graph.edges[edge]['weight']
        previous = np.zeros((len(graph.nodes), len(graph.nodes)))
        previous[:] = np.nan
        for i in range(len(graph.nodes)):
            previous[i][i] = i
        for edge in graph.edges:
            previous[edge[0]][edge[1]] = edge[0]
        for k in range(len(graph.nodes)):
            for i in range(len(graph.nodes)):
                for j in range(len(graph.nodes)):
                    if distance[i][j] > distance[i][k] + distance[k][j]:
                        distance[i][j] = distance[i][k] + distance[k][j]
                        previous[i][j] = previous[k][j]
        path = []
        node = target
        while node != start:
            path.append(node)
            node = int(previous[start][node])
        path.append(start)
        path.reverse()
        return path

    # NetworkX shortest path default algorithm
    def nxshp(graph, start, target):
        return nx.shortest_path(graph, source=start, target=target)

    # Bellman-Ford algorithm
    def bellman_ford(graph, start, target):
        distance = {}
        previous = {}
        for node in graph.nodes:
            distance[node] = math.inf
            previous[node] = None
        distance[start] = 0
        for i in range(len(graph.nodes)-1):
            for edge in graph.edges:
                if distance[edge[1]] > distance[edge[0]] + graph.edges[edge]['weight']:
                    distance[edge[1]] = distance[edge[0]] + graph.edges[edge]['weight']
                    previous[edge[1]] = edge[0]
        for edge in graph.edges:
            if distance[edge[1]] > distance[edge[0]] + graph.edges[edge]['weight']:
                raise ValueError("The graph contains a negative-weight cycle")
        path = []
        node = target
        while node != start:
            path.append(node)
            node = previous[node]
        path.append(start)
        path.reverse()
        return path

    # Johnson's algorithm
    def johnson(graph, start, target):
        graph.add_node(len(graph.nodes))
        for node in graph.nodes:
            if node != len(graph.nodes)-1:
                graph.add_edge(len(graph.nodes)-1, node, weight=0)
        distance = bellman_ford(graph, len(graph.nodes)-1, target)
        graph.remove_node(len(graph.nodes)-1)
        for edge in graph.edges:
            graph.edges[edge]['weight'] += distance[edge[0]] - distance[edge[1]]
        distance = dijkstra(graph, start, target)
        for i in range(len(distance)):
            distance[i] = distance[i] - distance[0] + distance[-1]
        return distance
        
    operations = {'dijkstra': dijkstra,
                  'floyd_warshall': floyd_warshall}
    reply = operations[algorithm](graph, start, target)
    if type_of_output == "path":
        return reply
    else:
        # Returns the length of the path
        length = 0
        for i in range(len(reply)-1):
            length += graph.edges[(reply[i], reply[i+1])]['weight']
        return length


In [11]:
#Generate a random graph
def graph_ran_generator():
    graph = nx.gnp_random_graph(25, 0.5, directed=True)
    for edge in graph.edges:
        graph.edges[edge]['weight'] = np.random.randint(1, 25)
    return graph

In [15]:
import doctest
doctest.testmod()




TestResults(failed=0, attempted=15)